**CNN İLE ANLAŞMAK**

Merhaba, bu yazımda kaggle içerisinde meşhur bir data olan Dogs vs Cats ile evrişimli sinir ağlarına ait bir örnek yapacağım.

Burada hem bir evrişimli sinir ağı için data nasıl hazırlanır hemde evrişimli sinir ağları nasıl çalışır bunları inceleyeceğim.

Aynı zamanda evrişimli sinir ağlarının bir kara kutu olup olmadığını elimden geldiğince anlatmaya çalışacağım.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Datayı hazırlamadan önce datayı nasıl çıkaracağız buna bakalım.

Data zip dosyası içinde bu nedenle önce zip dosyasından çıkarmak gerekiyor. Çıkan zip dosyası çalışma sayfasındaki output'un altında working klasörü içinde olacaktır.

In [ ]:
import zipfile

zip_files = ['test1', 'train']
for zip_file in zip_files:
    with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

Daha sonra train datasını liste şeklinde alalım ileride bu listeyi etiketleri belirken kullanacağız.

In [ ]:
image_path = "./train"
file_names = os.listdir(image_path)

In [ ]:
os.path.join(image_path, file_names[0])

Fotoğraf lisesinin bir örneği

In [ ]:
file_names[0].split(".")

Şimdi datayı hazırlamak için target ve train diye iki boş liste oluşturuyorum. Burada targets'lara fotoğrafın kedi veya köpek olduğu bilgisini vereceğim bunun için yukarıdaki file_names değişkeninden yararlanacağım. Train için ise sadece train klasörü ve peşine resim ismini ekleyeceğim bu kadar.

In [ ]:
targets = []
train = []

for file_name in file_names:
    target = file_name.split(".")[0]
    trains = os.path.join("./train", file_name)
    train.append(trains)
    targets.append(target)
    

data = pd.DataFrame()
data["image"] = train
data["target"] = targets

data.head()

Evet datamız yukarıdak gibi hazırlandı ve bu kadar. Gördüğünüz gibi resim üzerine çalışıyor olsak da datamız aslında resimlerin kaynakları yani nerede oldukları bilgisi ve bu resimlerin ne oldukları bilgisinden ibaret.

İkinci aşamada train içindeki resimleri train ve validation olarak bölme işlemine geldi. Burada klasik makine öğrenmesinden işlemi uyguluyorum ardından train datamın içindeki resimleri çeşitlendirmek için onları döndürme, büyütme, kaydırma gibi işlemleri tabi tutuyorum.

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size = 0.2)




from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   fill_mode = "nearest")

train_generator = train_datagen.flow_from_dataframe(dataframe = train_data,
                                                    x_col = "image",
                                                    y_col = "target",
                                                    target_size = (150, 150),
                                                    class_mode = "binary",
                                                    batch_size = 100)

train_generator isimli değişkende aslında train datamın ne olduğu bu data içirisindeki bağımlı ve bağımsız değişkenin ne olduğunu resimlerin boyutlarını ve bu resimlerin sadece iki çıktısı olduğunu söylüyorum.

Batch_size olarak 100 kullandım bu 20.000 elemanlı data üzerinden 100'er gruplar halinde data al demek.Neden datanın tamamı değilde sadece 100 tane? Aslıdan datanın tamamını kullanacağız fakat 100'lük gruplar halinde bu eğitim işlemimizin daha kısa sürede gerçekleşmesini sağlayacak elbette büyük batch_size ile çalışmak daha iyi model kurmamıza yarar sağlayabilir fakat performans olarak bu işlem uzun sürmesi muhtemeldir.

Resimlerde yaptğımız değişikliği görmek için bir örnek çizdirelim.

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

fnames = [os.path.join(image_path, fname) for fname in os.listdir(image_path)]

img_path = fnames[2]

img = image.load_img(img_path, target_size = (150, 150))

x = image.img_to_array(img)
x = x.reshape((1, ) + x.shape)
i = 0

for batch in train_datagen.flow(x, batch_size = 1):
    plt.figure()
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

plt.show()

Resim çeşitlendirmenin ne gibi bir faydası var?

> Aşırı uydurma, öğrenilecek çok az veri olduğunda modeli eğitmenin yeni verilere genelleştirmemesini olarak ortaya çıkar. Sonsuz veri olduğunda modeliniz var olan tüm veri dağılımlarını ortaya çıkarabilir ve asla aşırı uydurmaz. Veri seti çeşitlendirme eldeki verilerin birtakım döünüşümler ile gerçeğe yakın daha fazla eğitim verisi oluşturur. Amaç, eğitim esnasında modelinizi aynı resmi ikinci kez görmemesidir. Bu modelinizin daha çok örüntü aramasına ve daha iyi genelleştirmesini sağlar
                                                        
                                                        Francois Chollet, Deep Learning with Python sf147


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(dataframe = test_data,
                                                 x_col = "image",
                                                 y_col = "target",
                                                 target_size = (150, 150),
                                                 class_mode = "binary",
                                                 batch_size = 100)

Model için 4 katlı Conv2D hazırladım. Her katmandan sonra maxpooling işlemi yapıyorum. Katmanalardaki filtreler 3x3'lük activasyon fonksiyonu relu'yu tercih ettim. Son dense katmanında tahmin değerlerim 0 ile 1 arasında değerler alacağı için sigmoid fonksiyonunu kullandım. Compile aşamasında kayıp değeri için ikili sınıf olduğu için binarty_crossentropy kullandım. Model çıktısında yaklaşık 3.5 mio parametre var.

In [ ]:
from keras import layers
from keras import models
from keras import optimizers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = "relu", input_shape = (150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))

model.compile(loss = "binary_crossentropy",
             optimizer = optimizers.RMSprop(lr = 1e-4),
             metrics = ["acc"])

model.summary()

Modeli çalıştırmak.

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch = 200,
                             epochs = 100,
                             validation_data = test_generator,
                             validation_steps = 50)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)

plt.figure(figsize = (15, 6))

plt.plot(epochs, acc, "bo", label = "Eğitim Başarısı")
plt.plot(epochs, val_acc, "b", label = "Test Başarısı")
plt.title("Eğitim ve Test Başarısı")
plt.legend()

plt.figure(figsize = (15, 6))

plt.plot(epochs, loss, "bo", label = "Eğitim Kaybı")
plt.plot(epochs, val_loss, "b", label = "Doğrulama Kaybı")
plt.legend()

plt.show()

Evet veri çeşitlendirmesi yaparak modeli overfit etmekted kurtarmış gibi gözüküyoruz. Doğruluk oranı train ve test setimiz içinde %90'lara geldi. Burada önemli bir nokta hem veri çeşitliliği yapılması hemde model kurulurken maxpooling gibi parametre azaltma bir nevi modelin görüntü üzerinde daha geniş çalışmasını sağlama gibi etkenler olduğu açık.



Şimdi evrişimli sinir ağları bir kara kutu mu değil mi buna bakalım. Pek çok insan evrişimli sinir ağlarının arkasında mantığın bir sihir gibi görüyor fakat aslında konu öyle değil tek bir filtre çıktısı üzerinden eimden geldiğince anlatmaya çalışacağım.

In [ ]:
os.path.join(image_path, file_names[0])

Orjinal Resim

In [ ]:
img_path = "./train/cat.1650.jpg"

img = image.load_img(img_path, target_size = (150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis = 0)
img_tensor /= 255

plt.imshow(img_tensor[0])
plt.show()


In [ ]:
layer_output = [layer.output for layer in model.layers[:8]]
activation_model = models.Model(inputs = model.input, outputs = layer_output)
activations = activation_model.predict(img_tensor)

first_layer_activation = activations[0]

plt.matshow(first_layer_activation[0, :, :, 4], cmap = "viridis")

Şimdi yukarıdaki ilk resim kedinin orjinal resmi ikinci resim ise ilk katmandaki 4. kanalın çıktısı burada bakmamız gereken yer ikinci resimdeki açık yeşil olan yerler bunlar aslında birer dedektör. Konu aslında şu: açık yeşik yerlere baktığınız zaman gördüğünüz yerler kulaklar, burun çevresi, kedilerin etrafı. Aslında evrişimli sinir ağlarında herhangi bir sihir yok konu özellikle ilk katmanlar için tamamen kenar köşe bulma işi. Şunu unutmamak gerekir bilgisayar hiçbir zaman resimleri bizim gibi göremez resimleri onun için RGB kanallarındaki piksel değerlerinden ibaret yani o resim değil sayı görüyor! 

Peki sayı görerek nasıl kedi köpek tespiti yapabiliryor?

Şimdi yukarıdaki ilk resme bakın resmin büyük çoğunluğunu kediler kaplıyor değil mi ve bu kediler aslında birbirlerine benzer yani görünüşleri, boyutları, hatta renkleri işte sihir(!) burada başlıyor. Örneğin bilgisayarın resimdeki renk değerlerini (sayıları) yukarıdan aşağıya okuduğunu varsayalım en yukarı beyaz bir renk var muhtemelen bu bir kanepe veya yastık yani bir obje haliyle renk değerleride aynı resmin biraz aşağısına indiğinde kediler ile karşılaşıyor yani başka bir obje ama bilgisayar onları görerek farketmiyor renk değerlerinin değişimi ile şöyle bir yorum yapıyor: "Hmmm sanırım buraya kadar ki her şey aynıydı fakat şimdi buradan sonrası değişmeye başladı." Evet değişiyor çünkü renk değişiyor, sayılar değişiyor ve bilgisayar buna burada başka bir nesne var diyerek cevap veriyor bu yüzden kedilerin kulakları ve etrafları açık yeşil yani dedektöre takılıyor. Bunun gibi binlerce resim olunda bilgisayar bunları kolayca sınıflandırabiliyor.

Dedektöre takılma yerleri, büyüklükleri, kaba şekilleri vs gibi. Onun günün sonunda kedi veya köpek bulmak gibi bir derdi yok onun derdi sadece birbirine benzer objeleri aynı sınıfa atmak bu kadar.

Tüm katmanların görüntüsü
Kaynak:
* [https://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/5.4-visualizing-what-convnets-learn.ipynb](http://)

In [ ]:
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)

images_per_row = 16


for layer_name, layer_activation in zip(layer_names, activations):
   
    n_features = layer_activation.shape[-1]

    
    size = layer_activation.shape[1]

    
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()

Umarım bu çalışma sizin için faydalı olmuştur.

Tekrar görüşmek üzere!